In [ ]:
%matplotlib notebook
from math import cos, sin, pi
from matplotlib.figure import Figure
import ipywidgets as widgets
import odeanimate.plots
from odeanimate.domains import Interval
from odeanimate.vector import Vector2D, Vector3D
from odeanimate.jupyter import display_return

In [ ]:
def setup_figure(l_1, l_2, view=(25, 45)):
    rod_1_interval = Interval(-l_1 * (2**0.5), l_1 * (2**0.5))
    rod_2_interval = Interval(-l_2 * (2**0.5), l_2 * (2**0.5))
    z_interval = Interval(0, 3.5 * l_2)

    fig = Figure(figsize=(12, 6))
    gs = fig.add_gridspec(2, 4)

    ax_3d = fig.add_subplot(gs[:, :3], projection="odeanimate3D")
    ax_3d.set_limits(
        rod_1_interval,
        rod_2_interval,
        z_interval,
    )
    ax_3d.view_init(*view)

    ax_z = fig.add_subplot(gs[0, 3], projection="odeanimate")
    ax_z.set_limits(rod_1_interval)

    ax_t = fig.add_subplot(gs[1, 3], projection="odeanimate")
    ax_t.set_limits(rod_2_interval)

    return fig, (ax_3d, ax_z, ax_t)

In [ ]:
l_1, l_2 = 1.2, 2
fig, (ax_3d, ax_z, ax_t) = setup_figure(l_1, l_2)
fig

In [ ]:
def perform_calculation(l_1, l_2, theta, phi):
    # Calculate the positions
    # Full 3D
    origin, z = Vector3D(0, 0, 0), Vector3D(0, 0, 1)
    pivot = (1.5 * l_2) * z
    rod_1 = l_1 * Vector3D(cos(theta), sin(theta), 0)
    T = z ^ rod_1.direction
    rod_2 = l_2 * (cos(phi) * z + sin(phi) * T)
    rod_1.origin = pivot
    rod_2.origin = rod_1 + pivot
    # Projections
    # Top View
    rod_1_yx = rod_1.to_2D()
    rod_2_yx = rod_2.to_2D()
    rod_2_yx.origin = rod_1_yx
    # Tanget View
    rod_2_zt = Vector2D(rod_2 * T, rod_2 * z)

    return [pivot, rod_1, rod_2], [rod_1_yx, rod_2_yx], [rod_2_zt]

In [ ]:
def furuta_figure(theta, phi, l_1, l_2):
    fig, axes = setup_figure(l_1, l_2)
    calculated = perform_calculation(l_1, l_2, theta, phi)
    for ax, elems in zip(axes, calculated):
        ax.add(elems)
    return fig

In [ ]:
TAU = 6.35
widget_kwargs = dict(
    value=0,
    min=-TAU,
    max=TAU,
    step=0.005,
)
_theta = widgets.FloatSlider(
    description=r"$\theta$",
    **widget_kwargs,
)
_phi = widgets.FloatSlider(
    description=r"$\phi$",
    **widget_kwargs,
)
_rod_1_length = widgets.FloatSlider(
    description=r"$l_1$ Length",
    **{**widget_kwargs, "min": 1},
)
_rod_2_length = widgets.FloatSlider(
    description=r"$l_2$ Length",
    **{**widget_kwargs, "min": 1},
)
out = widgets.interactive_output(
    display_return(furuta_figure),
    {
        "theta": _theta,
        "phi": _phi,
        "l_1": _rod_1_length,
        "l_2": _rod_2_length,
    },
)
None

In [ ]:
widgets.VBox(
    [
        widgets.HBox(
            [
                widgets.VBox(
                    [
                        _theta,
                        _phi,
                    ]
                ),
                widgets.VBox(
                    [
                        _rod_1_length,
                        _rod_2_length,
                    ]
                ),
            ]
        ),
        out,
    ]
)